<a href="https://colab.research.google.com/github/Nidhicoconut/RAG./blob/main/gradpruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

Files already downloaded and verified


In [4]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified


In [5]:
import torch.nn as nn
import torch.nn.functional as F

# Define the ResNet block
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

# Define ResNet10
class ResNet10(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet10, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = BasicBlock(64, 64)
        self.layer2 = BasicBlock(64, 128, stride=2)
        self.layer3 = BasicBlock(128, 256, stride=2)
        self.layer4 = BasicBlock(256, 512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        return self.fc(out)


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet10(num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [7]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")

print("Training Finished!")


Epoch [1/20], Loss: 1.3832
Epoch [2/20], Loss: 0.9293
Epoch [3/20], Loss: 0.7417
Epoch [4/20], Loss: 0.6292
Epoch [5/20], Loss: 0.5484
Epoch [6/20], Loss: 0.4892
Epoch [7/20], Loss: 0.4441
Epoch [8/20], Loss: 0.4076
Epoch [9/20], Loss: 0.3723
Epoch [10/20], Loss: 0.3461
Epoch [11/20], Loss: 0.2507
Epoch [12/20], Loss: 0.2221
Epoch [13/20], Loss: 0.2117
Epoch [14/20], Loss: 0.1989
Epoch [15/20], Loss: 0.1919
Epoch [16/20], Loss: 0.1884
Epoch [17/20], Loss: 0.1777
Epoch [18/20], Loss: 0.1690
Epoch [19/20], Loss: 0.1631
Epoch [20/20], Loss: 0.1575
Training Finished!


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 89.65%


So initially accuracy is 90.36%

In [10]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=49270be3e7636e61d5c89784fd95073807b45bad40611c1609911bf3063d2f3f
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=f4d98e2ee3edae1e4e31f06e00115e407bdfec871372831415bfb2b55527e1a9
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [12]:
from fvcore.nn import FlopCountAnalysis, parameter_count_table

# Move model to device
model = model.to(device)

# Create a dummy input tensor (batch_size=1, 3 color channels, 32x32 CIFAR10 image)
dummy_input = torch.randn(1, 3, 32, 32).to(device)

# Compute FLOPs
flops = FlopCountAnalysis(model, dummy_input)
print(f"🔢 Total FLOPs: {flops.total()}")

# Compute Parameter Count
print(f"Model Parameters: \n{parameter_count_table(model)}")
print(f"Total Parameters: {sum(p.numel() for p in model.parameters())}")


layer1.shortcut


🔢 Total FLOPs: 254178304
Model Parameters: 
| name                   | #elements or shape   |
|:-----------------------|:---------------------|
| model                  | 4.9M                 |
|  conv1                 |  1.7K                |
|   conv1.weight         |   (64, 3, 3, 3)      |
|  bn1                   |  0.1K                |
|   bn1.weight           |   (64,)              |
|   bn1.bias             |   (64,)              |
|  layer1                |  74.0K               |
|   layer1.conv1         |   36.9K              |
|    layer1.conv1.weight |    (64, 64, 3, 3)    |
|   layer1.bn1           |   0.1K               |
|    layer1.bn1.weight   |    (64,)             |
|    layer1.bn1.bias     |    (64,)             |
|   layer1.conv2         |   36.9K              |
|    layer1.conv2.weight |    (64, 64, 3, 3)    |
|   layer1.bn2           |   0.1K               |
|    layer1.bn2.weight   |    (64,)             |
|    layer1.bn2.bias     |    (64,)             |
|  lay



1. Accuracy: 90.36%
2. FLOPs: 254178304
3. Total Parameters: 4903242



